# 목적
* 네이버 블로그에 검색어를 넣었을 때 나오는 블로그들을 크롤링
* 검색시 방법은 특정기간, 월별, 연별 검색에 따라 사용방법이 다소다름 (현재 연도최적화)

# 선행설치

In [1]:
#선행설치
# # 크롤링
# !pip install beautifulsoup4
# !pip install requests
# # 자연어처리
# !pip install konlpy (JDK 설치가 되어있어야함 )
# # 워드클라우드
# !pip install wordcloud

In [2]:
# import

from tqdm import tqdm_notebook  # 진행과정 시각화
from datetime import timedelta  # 시간날짜

import re
import os
import datetime
import pandas as pd
import numpy as np
import gc

import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup  # 크롤링


from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

In [3]:
def del_outword(string):
    '''
    크롤링 후 이모티콘, \u200b과 같은 문자가 아닌 것 제거
    #이모티콘 제거 [출처](https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)
    '''
    #이모지제거
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~"^_\n\r#\ufeff\u200d\u200b\u7643\ufffd\u682e\u62c4]+')
    
    string = emoji_pattern.sub(r'',string)
    string = han.sub(r'',string)
    
    return string

In [4]:
def time_change(x):
    '''
    검색결과에 있는 문자를 날짜로 변경
    '''
    min_pattern = re.compile('[0-9]+'+"분 전")
    hour_pattern = re.compile('[0-9]+'+"시간 전")
    day_pattern = re.compile('[0-9]+'+"일 전")

    today = datetime.datetime.today().date()
    # 일자
    try:
        d = re.findall(day_pattern, x)[0][0]
        x = today - timedelta(days=int(d))
    except:
        pass
    # 시간
    try:
        d = re.findall(hour_pattern, x)[0]
        x = today
    except:
        pass
    # 분
    try:
        h = re.findall(min_pattern, x)[0]
        x = today
    except:
        pass

    if x == '어제':
        x = today - timedelta(days=1)

    elif type(x) == str:
        x = datetime.datetime.strptime(x.replace('.', '-')[:-1], '%Y-%m-%d')
    #모든일자를 1일로 통일
    x = datetime.datetime.strptime(x.strftime('%Y-%m')+"-01","%Y-%m-%d")
    return x

In [5]:
def naver_blog_crawling(keyword, start_num=1, end_num=101, date_option=0, date_from='', date_to='', save=True):
    '''네이버 블로그 크롤링 함수
    네이버 블로그 검색결과를 크롤링하며, 1페이지당 10개씩을 검색한다
    
    Parameters
    ----------
    keyword(string) : 검색하고 싶은 키워드를 넣는다 "keyword +필수어" 형태로 필수단어 추가 가능
    start_num(int) : (default = 1)  시작할 위치, 1로 끝나는 단위 추천
    end_num(int) : (default = 101) 끝나는 위치, 1로 끝나는 단위 추천
    date_option(int) : (default = 0) 주어지는 숫자에 의해 검색방법이 변경됨
                         0 : 전체, 2 : 1일, 3 : 1주, 4 : 1개월, 6 : 6개월, 7 : 1년, 8 : 기간지정
    date_from(YYYYMMDD) : (default = "") date_option이 8일때 사용 검색 시작일자를 지정
    date_to(YYYYMMDD) : (default = ""), date_option이 8일때 사용 검색 마지막일자를 지정
    save(bool) : (default = True)csv로 저장 여부 결정
    '''
    # url 찾는 패턴
    pattern = re.compile('href="'+'[A-z0-9\:\/\&\;\.\?\=]+')

    # 저장위치
    postdates = []
    strings = []
    urls = []
    titles = []
    output_error = []
    connection_error = []
    count = 1

    # keyword와 시작넘버만 바꾸면서 진행하게끔
    base_url = 'https://search.naver.com/search.naver?date_from={date_from}&date_option={date_option}&date_to={date_to}&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post&start={start}'

    try:
        # check length
        search_list = base_url.format(keyword=keyword, start=start_num,
                                      date_option=date_option, date_from=date_from, date_to=date_to)
        response = requests.get(search_list)
        soup = BeautifulSoup(response.content, 'lxml')
        max_searched = soup.select("div.section_head")[0].text.split('/')[1]
        max_searched = int(re.sub('[가-힣 ,]+', "", max_searched))
        end_num = max_searched
    except:
        pass

    # for문 돌려야하는 부분(각 검색결과의 시작은 1이고, 10개씩 보여짐,)
    for i in tqdm_notebook(range(start_num, end_num+10, 10), desc = 'section'):
        search_list = base_url.format(keyword=keyword, start=i,
                                      date_option=date_option, date_from=date_from, date_to=date_to)
        response = requests.get(search_list)

        if response.status_code is 200:
            # 1번 검색시 10개의 결과가 출력 따라서 section은 총 10개
            soup = BeautifulSoup(response.content, 'lxml')
            sections = soup.findAll('li', attrs={'class': 'sh_blog_top'})

            for section in sections:
                try:
                    # href부분만 가져오기 어려워서 정규표현식으로 검색
                    url = re.findall(pattern, str(section))[0].replace(
                        '?Redirect=Log&amp;logNo=', '/').replace('href="', '')
                    title = section.select_one('a.txt84').text
                    date = section.select_one('dd.txt_inline').text.strip()

                    # 블로그 url안에 들어가기(아직 크롤링불가)
                    get_blog_post_content_code = requests.get(url)
                    get_blog_post_content_text = get_blog_post_content_code.text
                    get_blog_post_content_soup = BeautifulSoup(
                        get_blog_post_content_text, 'lxml')

                    # 크롤링가능한 url에 접속
                    real_blog_post_url = "http://blog.naver.com" + \
                        get_blog_post_content_soup.select('#mainFrame')[
                            0].get('src')
                    get_real_blog_post_content_code = requests.get(
                        real_blog_post_url)
                    get_real_blog_post_content_text = get_real_blog_post_content_code.text
                    get_real_blog_post_content_soup = BeautifulSoup(
                        get_real_blog_post_content_text, 'lxml')

                    # url (에러나면 위에서부터 에러남)
                    urls.append(real_blog_post_url)
                    # 블로그명
                    titles.append(title)
                    # 날짜
                    postdates.append(date)

                    # 본문  (postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함)
                    try:
                        blog_post_content = get_real_blog_post_content_soup.select(
                            'div#postViewArea')
                        if len(blog_post_content) == 0:
                            blog_post_content = get_real_blog_post_content_soup.select(
                                'div.se-main-container')
                            if len(blog_post_content) == 0:
                                blog_post_content = get_real_blog_post_content_soup.select(
                                    'div.se_component_wrap.sect_dsc.__se_component_area')

                        string = ""
                        for sentence in blog_post_content[0].stripped_strings:
                            string += " "+sentence.replace('\xa0', " ")
                            # 비언어 텍스트제거
                            string = del_outword(string)
                        # 공백에러대처
                        blank_check = string.replace(" ", "")
                        if len(blank_check) == 0:
                            strings.append(["X"])
                        else:
                            strings.append([string])
                        count += 1
                    except:
                        strings.append(["X"])
                        output_error.append(count)
                        count += 1

                except Exception as ex:
                    # print('가져오기에러 {num}번째'.format(num = count),ex)
                    output_error.append(count)
                    count += 1
                    pass
        else:
            # print('연결오류 {num}번째'.format(num = count),response.status_code)
            connection_error.append(count)
            count += 1

    out_length = len(output_error+connection_error)
#     real_length = end_num-start_num+10-out_length

    crawling_df = pd.DataFrame(
        {"post_dates": postdates, "title": titles, "full_text": strings, "url": urls})
    crawling_df['post_dates'] = crawling_df['post_dates'].apply(
        lambda x: time_change(x))
    # 중복제거
    crawling_df.drop_duplicates("url", inplace=True)
    crawling_df.reset_index(drop=True, inplace=True)
    real_length = crawling_df.shape[0]

    # 결과출력
#     print("검색한 길이:", end_num-start_num+10)
#     print("제외된 길이:", out_length)
#     print("검색된 길이:", real_length)

    if save == True:
        today = datetime.datetime.today().date()
        str(today)
        os.makedirs(
            './output/크롤링/{keyword}'.format(keyword=keyword), exist_ok=True)
        # 저장이름 "기간_키워드_길이_날짜" ex) 20160501~20160531_물치리 +강원도_9_2020-05-07
        crawling_df.to_csv(f"./output/크롤링/{keyword}/{date_from}~{date_to}_{keyword}_{real_length}_{today}.csv", encoding='utf-8', index=False)
    return crawling_df, real_length

In [6]:
def make_year_list(start_year,end_year,year_gap = 1):
    '''
    년도별로 검색할때 사용 
    '''
    year_list = []
    for year in range(start_year,end_year+1,year_gap):
        year_list.append(str(year))
    if str(end_year) not in year_list:
        year_list.append(str(end_year))
    #start_days
    date_from = list(map(lambda year: year+'0101',year_list))
    #end_days 
    date_to = list(map(lambda year: year+'1231',year_list))
    return date_from,date_to

In [7]:
def make_month_list(start_year,end_year,month_gap = 1,year_gap = 1):
    '''
    월별로 검색할때 사용
    '''
    #일자
    month_gap = month_gap*100
    dates_from = []
    dates_to = []
    for date in range(101,1202,month_gap):
        dates_from.append(date)
    dates_from = list(map(lambda x : '{:0>4}'.format(str(x)),dates_from))

    for date in range(131,1232,month_gap):
        dates_to.append(date)
    dates_to = list(map(lambda x : '{:0>4}'.format(str(x)),dates_to))

    date_from = []
    date_to = []
    for year in range(start_year,end_year+1,year_gap):
        #시작일자
        for date in dates_from:
            date_from.append(str(year)+date)
        #마지막일자
        for date in dates_to:
            date_to.append(str(year)+date)
    if year != end_year:
        #시작일자
        for date in dates_from:
            date_from.append(str(year)+date)
        #마지막일자
        for date in dates_to:
            date_to.append(str(year)+date)
    return date_from,date_to

### 검색방법 및 불용어설정

In [8]:
# 특정기간 설정
word_list = ["물치","설악해수욕장","낙산","오산","수산항","동호","하조대","하광정리","기사문","동산","인구","광진","남애"]
mustword = "양양"

start_num = 1
end_num = 701
date_option = 8
date_from = "20150101"
date_to = "20151231"
today = datetime.datetime.today().date()

In [9]:
#연도별 설정
word_list =["","강릉","양양","속초","경포해수욕장","속초해수욕장","낙산해수욕장"]
mustword = "강원도"
start_num = 1
end_num = 991
date_option = 8
start_year = 2010
end_year = 2019
gap = 5
today = datetime.datetime.today().date()
#날짜범위함수 
date_from,date_to = make_year_list(start_year,end_year,gap)

In [18]:
#월별 설정
word_list =["오산리"]
mustword = "양양"
start_num = 1
end_num = 991
date_option = 8
start_year = 2010
end_year = 2019
month_gap = 1
year_gap = 1
today = datetime.datetime.today().date()
#날짜범위함수 
date_from,date_to = make_month_list(start_year,end_year,month_gap,year_gap)

In [19]:
#불용어
sw = list(pd.read_excel("stopword(cp949).xlsx",encoding = 'cp949')['불용어']) #불용어 불러오기

In [20]:
def searching_all(word_list, mustword, start_num, end_num, date_option, date_from, date_to):
    #키워드별 
    for i in tqdm_notebook(range(0,len(word_list)), desc = 'Total'):
        keyword = word_list[i] + " +" + mustword
        print('{num}번째/{total_num}번째 {keyword}'.format(num = i+1, total_num = len(word_list),keyword = keyword))
        #날짜가 연별,월별인경우 
        if type(date_from) == list:
            for date_num in tqdm_notebook(range(0,len(date_from)),desc = "Time"):
                date_from_one = date_from[date_num]
                date_to_one = date_to[date_num]
                crawling_df, real_length = naver_blog_crawling(keyword, start_num, end_num, date_option, date_from_one, date_to_one)
#                 output_graph(okt,crawling_df,keyword,real_length,sw,date_from_one,date_to_one,unique=False)
        #단일날짜의 경우 
        else:
            crawling_df, real_length = naver_blog_crawling(keyword, start_num, end_num, date_option, date_from, date_to)
#             output_graph(okt,crawling_df,keyword,real_length,sw,date_from,date_to,unique=False)
        gc.collect()

In [21]:
searching_all(word_list = word_list, mustword = mustword, start_num = start_num, end_num = end_num, date_option = date_option, date_from = date_from, date_to = date_to)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


1번째/1번째 오산리 +양양


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
